In [1]:
import cv2
import mediapipe as mp
import handtrackingModule as htm 
import time 
import numpy as np
import pyautogui
from pynput.mouse import Controller, Button

In [4]:
wcam , hcam = 1280,720
framered = 100
smoothening = 7

ptime = 0
plocx, plocy = 0,0
clocx, clocy = 0,0

cap = cv2.VideoCapture(0)
cap.set(3,wcam)
cap.set(4,hcam)
detector = htm.handDetector(maxHands=1)
mouse = Controller()
wscreen, hscreen = mouse.screen_size if hasattr(mouse, 'screen_size') else (1920,1080)
#wscreen , hscreen = pyautogui.size()

while True:
    #find hands landmarks
    success, img = cap.read()
    img = detector.findHands(img)
    lmslist,bbox = detector.findPosition(img)
        
    # get the tip of the index and middle finger
    if len(lmslist) != 0:
        x1,y1 = lmslist[8][1:]
        x2,y2 = lmslist[12][1:]
        
    #check which finger is up
    fingers = detector.fingersUp()
    cv2.rectangle(img,(framered,framered),(wcam-framered,hcam-framered),(255,0,0),2)
    
    #if only the index finger is up: moving mode
    if len(fingers) > 1 and fingers[1] == 1 and fingers[2] == 0:
        #convert coordinates
        x3 = np.interp(x1,(0,wcam),(0,wscreen))
        y3 = np.interp(y1,(framered,hcam-framered),(0,hscreen))
        
        #smooth values
        clocx = plocx + (x3 - plocx) / smoothening
        clocy = plocy + (y3 - plocy) / smoothening
        
        #move mouse 
        #pyautogui.moveTo(wscreen - clocx,clocy)
        mouse.position = (wscreen - clocx,clocy)
        cv2.circle(img,(x1,y1),15,(255,0,0),cv2.FILLED)
        plocx, plocy = clocx, clocy
    
    # if both fingers are up: clicking mode
    if len(fingers) > 1 and fingers[1] == 1 and fingers[2] == 1:
        #find distance between fingers
        length, img, lineinfo = detector.findDistance(8,12,img)
        #click mouse if distance short
        if length < 40:
            cv2.circle(img,(lineinfo[4],lineinfo[5]),15,(255,0,0),cv2.FILLED)
            #pyautogui.click()
            mouse.click(Button.left, 1)
            time.sleep(0.5) 
            
        
             

    
    ctime = time.time()
    fps = 1/(ctime-ptime)
    ptime = ctime
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 0), 3)
        

    cv2.imshow("image",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
